In [13]:
pip install pandas pyodbc sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pyodbc
 
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-29912U2\MSSQLSERVER1;'
    'DATABASE=pipelines;'
    'UID=sa;'
    'PWD=Dhiva@2003;'
    'TrustServerCertificate=yes;'
)
 
print("Database Connected successfully!")

Database Connected successfully!


In [15]:
import pandas as pd 
customer = pd.read_csv(r"C:\Users\Dhivakar\Downloads\us_customer_data 1.csv")
print(customer.head())

   customer_id              name                        email         phone  \
0            1     Michelle Kidd           vayala@example.net  619-723-4258   
1            2       Brad Newton  taylorcatherine@example.net  537-674-1158   
2            3      Larry Torres         dsanchez@example.net  810-256-4505   
3            4    Kimberly Price    jessicaknight@example.com  423-222-9779   
4            5  Matthew Phillips        qwilliams@example.com  220-763-3522   

                                             address registration_date  \
0                        USNS Santiago, FPO AE 80872        25-01-2025   
1  38783 Oliver Street, West Kristenborough, MT 9...        13-07-2023   
2     6845 Steele Turnpike, West Erikabury, UT 37487        18-08-2023   
3         1631 Alexis Meadows, Lake Amanda, CA 75179        08-12-2024   
4  2274 Williams Heights Suite 895, Andersonhaven...        03-02-2024   

  loyalty_status  
0           Gold  
1         Silver  
2         Bronze  
3   

In [19]:
transaction = pd.read_csv(r"C:\Users\Dhivakar\pipelines\transaction_data.csv")
transaction.head()

,transaction_id,customer_id,amount,transaction_date,product_category,payment_method,store_location
0,1,565,2992.47,2025-03-10 01:20:54,Sports,Debit Card,New York
1,2,323,2041.87,2025-01-02 15:24:19,Clothing,Cash,New York
2,3,398,107.35,2025-02-16 03:49:01,Beauty,Debit Card,Online
3,4,19,NaN,2025-04-30 15:26:23,Sports,Debit Card,Los Angeles
4,5,547,3063.28,2025-06-14 04:28:53,Clothing,PayPal,Los Angeles


In [21]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import pandas as pd
 

connection_url = URL.create(
    "mssql+pyodbc",
    username="sa",
    password="Dhiva@2003",
    host="DESKTOP-29912U2\MSSQLSERVER1",
    database="pipelines",
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "TrustServerCertificate": "yes"
    }
)
 
engine = create_engine(connection_url)
 
customer.to_sql("customers", con=engine, if_exists="replace", index=False)
transaction.to_sql("transactions", con=engine, if_exists="replace", index = False)
 
print("Data uploaded successfully to SQL Server!")

Data uploaded successfully to SQL Server!


In [22]:
orders = pd.read_csv(r"C:\Users\Dhivakar\pipelines\order_data 1.csv")
orders.head()

,order_id,customer_id,order_date,order_amount,order_status,product_category
0,892a07a4-d252-4775-85e0-73077143e1c6,966,2024-11-27,317.64,Cancelled,Home & Garden
1,ae160758-e187-47b2-9350-032f88f55491,345,2023-03-27,645.87,Completed,Home & Garden
2,7c50456e-6123-45cc-aa19-128bef3754d6,503,2024-03-31,880.86,Pending,Clothing
3,c788b56b-3716-4cd9-a827-d4dc401ba00c,385,2023-08-09,876.83,Cancelled,Home & Garden
4,925ab5b1-adb1-4302-a70b-1c2db724e02b,817,2023-04-19,264.53,Pending,Home & Garden


In [23]:
orders.to_sql('orders', con = engine, if_exists='replace', index= False)

255

In [25]:
customers_sql = pd.read_sql("Select * from customers", con=engine)
customers_sql.head()

,customer_id,name,email,phone,address,registration_date,loyalty_status
0,1,Michelle Kidd,vayala@example.net,619-723-4258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold
1,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver
2,3,Larry Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze
3,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold
4,5,Matthew Phillips,qwilliams@example.com,220-763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold


In [26]:
orders_sql = pd.read_sql("Select * from orders", con=engine)
orders_sql.head()

,order_id,customer_id,order_date,order_amount,order_status,product_category
0,892a07a4-d252-4775-85e0-73077143e1c6,966,2024-11-27,317.64,Cancelled,Home & Garden
1,ae160758-e187-47b2-9350-032f88f55491,345,2023-03-27,645.87,Completed,Home & Garden
2,7c50456e-6123-45cc-aa19-128bef3754d6,503,2024-03-31,880.86,Pending,Clothing
3,c788b56b-3716-4cd9-a827-d4dc401ba00c,385,2023-08-09,876.83,Cancelled,Home & Garden
4,925ab5b1-adb1-4302-a70b-1c2db724e02b,817,2023-04-19,264.53,Pending,Home & Garden


In [27]:
customers_sql[['name']].loc[132]

name    Elizabeth Williams MD
Name: 132, dtype: object

In [30]:
#split full name into two parts first and last 
prefixes = r'^(Mr\.|Mrs\.|Miss|Ms\.|Dr\.)\s*'
suffixes = r'\s*(Jr\.|Sr\.|II|III|DDS|PhD|MD|MBA|DVM|Esq\.?)$'
customers_sql['name'] = customers_sql['name'].str.replace(prefixes, '', regex=True)
customers_sql['name'] = customers_sql['name'].str.replace(suffixes, '', regex=True)
customers_sql[['name']].loc[100]

name    Amy Graham
Name: 100, dtype: object

In [31]:
customers_sql[['first_name', 'last_name']] = customers_sql['name'].str.strip().str.split(' ', n=1, expand=True)
print(customers_sql[['first_name', 'last_name']])

    first_name last_name
0     Michelle      Kidd
1         Brad    Newton
2        Larry    Torres
3     Kimberly     Price
4      Matthew  Phillips
..         ...       ...
995      Jerry  Mcdaniel
996       Jodi   Simpson
997    Crystal     Brown
998    Gregory    Duarte
999      Jason     Walls

[1000 rows x 2 columns]


In [33]:
customers_sql.head()

,customer_id,name,email,phone,address,registration_date,loyalty_status,first_name,last_name
0,1,Michelle Kidd,vayala@example.net,619-723-4258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,Michelle,Kidd
1,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,Brad,Newton
2,3,Larry Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,Larry,Torres
3,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,Kimberly,Price
4,5,Matthew Phillips,qwilliams@example.com,220-763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold,Matthew,Phillips


In [36]:
customers_sql['country_code'] = customers_sql['address'].str.slice(-8,-6)
customers_sql.head(3)

,customer_id,name,email,phone,address,registration_date,loyalty_status,first_name,last_name,country_code
0,1,Michelle Kidd,vayala@example.net,619-723-4258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,Michelle,Kidd,AE
1,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,Brad,Newton,MT
2,3,Larry Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,Larry,Torres,UT


In [37]:
customers_sql['phone'] = customers_sql['phone'].str.replace('-','')
customers_sql.head(3)

,customer_id,name,email,phone,address,registration_date,loyalty_status,first_name,last_name,country_code
0,1,Michelle Kidd,vayala@example.net,6197234258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,Michelle,Kidd,AE
1,2,Brad Newton,taylorcatherine@example.net,5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,Brad,Newton,MT
2,3,Larry Torres,dsanchez@example.net,8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,Larry,Torres,UT


In [46]:
#drop the name and replace it with First and Last Name.
customers_sql = customers_sql.drop('name', axis = 1)

In [43]:
customers_sql.head()

,customer_id,email,phone,address,registration_date,loyalty_status,first_name,last_name,country_code
0,1,vayala@example.net,6197234258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,Michelle,Kidd,AE
1,2,taylorcatherine@example.net,5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,Brad,Newton,MT
2,3,dsanchez@example.net,8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,Larry,Torres,UT
3,4,jessicaknight@example.com,4232229779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,Kimberly,Price,CA
4,5,qwilliams@example.com,2207633522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold,Matthew,Phillips,OR


In [47]:
customers_sql.head(3)

,customer_id,email,phone,address,registration_date,loyalty_status,first_name,last_name,country_code
0,1,vayala@example.net,6197234258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,Michelle,Kidd,AE
1,2,taylorcatherine@example.net,5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,Brad,Newton,MT
2,3,dsanchez@example.net,8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,Larry,Torres,UT


In [48]:
#Rearranging the order of columns
customers_sql = customers_sql[['customer_id', 'first_name', 'last_name', 'email', 'phone', 'address', 'registration_date', 'loyalty_status', 'country_code']]
customers_sql.head(3)

,customer_id,first_name,last_name,email,phone,address,registration_date,loyalty_status,country_code
0,1,Michelle,Kidd,vayala@example.net,6197234258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,AE
1,2,Brad,Newton,taylorcatherine@example.net,5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,MT
2,3,Larry,Torres,dsanchez@example.net,8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,UT


In [51]:
#Formatting Phone numbers with extracting country codes prepend.

def format_phone(row):
    phone = str(row['phone']).strip()
    phone_digits = ''.join(filter(str.isdigit, phone)) 

    if len(phone_digits) != 10:
        return 'Invalid'
    
    code = row['country_code']
    if code == 'IN':
        return '+91 ' + phone_digits
    elif code == 'AE':
        return '+971 ' + phone_digits
    elif code == 'FM':
        return '+691 ' + phone_digits 
    elif code == 'PW':
        return '+680 ' + phone_digits
    else:
        return '+1 ' + phone_digits
    
customers_sql['phone'] = customers_sql.apply(format_phone, axis = 1)

In [52]:
customers_sql.head(4)

,customer_id,first_name,last_name,email,phone,address,registration_date,loyalty_status,country_code
0,1,Michelle,Kidd,vayala@example.net,+971 6197234258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,AE
1,2,Brad,Newton,taylorcatherine@example.net,+1 5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,MT
2,3,Larry,Torres,dsanchez@example.net,+1 8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,UT
3,4,Kimberly,Price,jessicaknight@example.com,+1 4232229779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,CA


In [53]:
#adding customer_tier

def format_tier(row):

    tier_name = str(row['loyalty_status'])

    if tier_name == "Gold":
        return '2'
    elif tier_name == 'Silver':
        return '1'
    else:
        return '0'

customers_sql['customer_tier'] = customers_sql.apply(format_tier, axis = 1)

In [54]:
customers_sql.head(3)

,customer_id,first_name,last_name,email,phone,address,registration_date,loyalty_status,country_code,customer_tier
0,1,Michelle,Kidd,vayala@example.net,+971 6197234258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,AE,2
1,2,Brad,Newton,taylorcatherine@example.net,+1 5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,MT,1
2,3,Larry,Torres,dsanchez@example.net,+1 8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,UT,0


In [55]:
#unified view of customers and orders
unified_customers = pd.merge(customers_sql, orders_sql, on='customer_id', how='inner')
unified_customers.head(3)

,customer_id,first_name,last_name,email,phone,address,registration_date,loyalty_status,country_code,customer_tier,order_id,order_date,order_amount,order_status,product_category
0,2,Brad,Newton,taylorcatherine@example.net,+1 5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,MT,1,209c024e-6c32-49fa-acd1-2501bd19c1cd,2023-10-30,486.53,Pending,Toys
1,2,Brad,Newton,taylorcatherine@example.net,+1 5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,MT,1,3f263091-da18-48a6-8b00-c179e1b1a81a,2024-08-29,781.22,Pending,Books
2,3,Larry,Torres,dsanchez@example.net,+1 8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,UT,0,63d7870e-617e-4f49-ae9b-b893e635e43f,2023-04-05,454.07,Pending,Toys


In [58]:
unified_customers.duplicated().sum()

np.int64(0)

In [59]:
#Again loading the unified customers to sql server
unified_customers.to_sql('customers_orders', con = engine, if_exists='replace', index = False)
print('uploaded successfully')

uploaded successfully
